In [16]:
# # load dotenv + imports for retriever tool
from getpass import getpass
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma 
from langchain.tools import tool
import langchainhub as hub
import chromadb
from langchain_google_genai import GoogleGenerativeAIEmbeddings


api_key = os.getenv("CHROMA_API_KEY") or getpass("Paste CHROMA_API_KEY: ")
tenant = os.getenv("CHROMA_TENANT") or input("Enter CHROMA_TENANT: ")
database = os.getenv("CHROMA_DATABASE") or input("Enter CHROMA_DATABASE: ")
gemini_key = os.getenv("GEMINI_API_KEY") or getpass("Enter GEMINI_API_KEY: ")

# Load environment variables from chromaDB --CloudClient
client = chromadb.CloudClient(
    api_key=api_key,
    tenant=tenant,
    database=database
)
collection = client.get_collection(name="nice_org")

# connecting embeddings with Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",gemini_api_key=gemini_key)

vectorstore = Chroma(
    client=client,
    collection_name="nice_org",
    embedding_function=embeddings
)

print("Connection successful")

ValidationError: 1 validation error for GoogleGenerativeAIEmbeddings
  Value error, API key required for Gemini Developer API. Provide api_key parameter or set GOOGLE_API_KEY/GEMINI_API_KEY environment variable. [type=value_error, input_value={'model': 'models/text-em...SU3KJE9HSeg1-0nzzcvmW0'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [ ]:
import os 
print(f"Chroma Key Found: {os.getenv('CHROMA_API_KEY') is not None}")